In [44]:
from __future__ import print_function

import numpy as np
import pandas as pd
from keras.models import Model
from keras.utils import np_utils
import keras 
from keras.callbacks import ReduceLROnPlateau

# read data and preprocess data

In [45]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
foder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

#flist  = ['MiddlePhalanxTW']
#for each in flist:
each = 'Adiac'

fname = each
x_train, y_train = readucr(foder+fname+'/'+fname+'_TRAIN')
x_test, y_test = readucr(foder+fname+'/'+fname+'_TEST')

###################################################################################################################
'''模型变量'''
nb_epochs = 40
nb_feature = 0
nb_classes = len(np.unique(y_test))
nb_feature = x_train.shape[1]
batch_size = min(x_train.shape[0]/10, 16)
####################################################################################################################

y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean)/(x_train_std) 
x_test = (x_test - x_train_mean)/(x_train_std)

x_train = x_train.reshape(x_train.shape + (1,1,))
x_test = x_test.reshape(x_test.shape + (1,1,))

In [46]:
print("# x_train.shape: ",x_train.shape)
print("# x_test.shape: ",x_test.shape)

print("# Y_train.shape: ",Y_train.shape)
print("# Y_test.shape: ",Y_test.shape)

# x_train.shape:  (390, 176, 1, 1)
# x_test.shape:  (391, 176, 1, 1)
# Y_train.shape:  (390, 37)
# Y_test.shape:  (391, 37)


# Model_Sequential模型

In [ ]:
'''CNN'''

In [59]:
import numpy as np
np.random.seed(1337)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

# 全局变量
batch_size = batch_size
nb_classses = nb_classes
epochs = nb_epochs

# input image dimensions
img_rows, img_cols = x_train.shape[1], x_train.shape[2]
# number of convolutional filters to use
nb_filters_1 = 128
nb_filters_2 = 256
nb_filters_3 = 128
# size of pooling area for max pooling
pool_size = (3,1)
# convolution kernel size
kernel_size_1 = (8,1)
kernel_size_2 = (5,1)
kernel_size_3 = (3,1)

input_shape = (x_train.shape[1:])
print(input_shape)

(176, 1, 1)


In [61]:
model = Sequential()  

model.add(Convolution2D(nb_filters_1, (kernel_size_1[0], kernel_size_2[1]), padding='same', input_shape=input_shape)) # 卷积层1  
model.add(Activation('relu')) #激活层  

model.add(Convolution2D(nb_filters_2, (kernel_size_2[0], kernel_size_2[1]))) #卷积层2  
model.add(Activation('relu')) #激活层

model.add(Convolution2D(nb_filters_3, (kernel_size_3[0], kernel_size_3[1]))) #卷积层3  
model.add(Activation('relu')) #激活层

#model.add(MaxPooling2D(pool_size=pool_size)) #池化层  


model.add(Reshape((176, nb_filters_3)))
model.add(LSTM(8))


model.add(Dropout(0.25)) #神经元随机失活  
model.add(Flatten()) #拉成一维数据  
model.add(Dense(128)) #全连接层1  
model.add(Activation('relu')) #激活层  
model.add(Dropout(0.5)) #随机失活  
model.add(Dense(nb_classes)) #全连接层2  
model.add(Activation('softmax')) #Softmax评分  
  
#编译模型  
model.compile(loss='categorical_crossentropy',  
              optimizer='adadelta',  
              metrics=['accuracy'])  

#训练模型  
model.fit(x_train, Y_train, batch_size=batch_size, epochs=epochs,  
          verbose=1, validation_data=(x_test, Y_test))  
#评估模型  
score = model.evaluate(x_test, Y_test, verbose=0)  
print('Test score:', score[0])  
print('Test accuracy:', score[1])  

ValueError: Input 0 is incompatible with layer lstm_5: expected ndim=3, found ndim=4

# Model_函数式模型

In [55]:
from __future__ import print_function
 
from keras.models import Model
from keras.utils import np_utils
import numpy as np
import pandas as pd
import keras 
from keras.layers import LSTM
from keras.callbacks import ReduceLROnPlateau 

filters_1 = 128
filters_2 = 256
filters_3 = 128

kernel_size_1 = (8,1)
kernel_size_2 = (5,1)
kernel_size_3 = (3,1)

x = keras.layers.Input(x_train.shape[1:])

#    drop_out = Dropout(0.2)(x)
conv1 = keras.layers.Conv2D(128, (8, 1), border_mode='same')(x)
conv1 = keras.layers.normalization.BatchNormalization()(conv1)
conv1 = keras.layers.Activation('relu')(conv1)

print("##################################")
print("输出测试 conv1 —— feature map: ")
print(conv1)
print("##################################")

#    drop_out = Dropout(0.2)(conv1)
conv2 = keras.layers.Conv2D(256, (5, 1), border_mode='same')(conv1)
conv2 = keras.layers.normalization.BatchNormalization()(conv2)
conv2 = keras.layers.Activation('relu')(conv2)

print("输出测试 conv2 —— feature map: ")
print(conv2)
print("##################################")

#    drop_out = Dropout(0.2)(conv2)
conv3 = keras.layers.Conv2D(128, (3, 1), border_mode='same')(conv2)
conv3 = keras.layers.normalization.BatchNormalization()(conv3)
conv3 = keras.layers.Activation('relu')(conv3)

print("输出测试 conv3 —— feature map: ")
print(conv3)
print("###################################")

full = keras.layers.Reshape(conv3.shape[1], conv3.shape[2], conv3.shape[3])(conv3)
full = LSTM(8, input_shape=(full.shape[1], full.shape[2]), return_sequences=True)(conv3)

#full = keras.layers.pooling.GlobalAveragePooling2D()(conv3)

out = keras.layers.Dense(nb_classes, activation='softmax')(full)

model = Model(input=x, output=out)
 
optimizer = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
 
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                  patience=50, min_lr=0.0001) 
hist = model.fit(x_train, Y_train, batch_size=batch_size, nb_epoch=nb_epochs,
          verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
#Print the testing results which has the lowest training loss.
log = pd.DataFrame(hist.history)
print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (8, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 1), padding="same")`


##################################
输出测试 conv1 —— feature map: 
Tensor("activation_49/Relu:0", shape=(?, 176, 1, 128), dtype=float32)
##################################
输出测试 conv2 —— feature map: 
Tensor("activation_50/Relu:0", shape=(?, 176, 1, 256), dtype=float32)
##################################
输出测试 conv3 —— feature map: 
Tensor("activation_51/Relu:0", shape=(?, 176, 1, 128), dtype=float32)
###################################


TypeError: __init__() takes 2 positional arguments but 4 were given

# 新的 1D卷积+1LSTM

In [ ]:
from keras.models import Model
from keras.layers import Input, PReLU, Dense, LSTM, multiply, concatenate, Activation
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout

In [ ]:
def generate_model():
    ip = Input(shape=(1, 176))  #  timesteps, nb_feature/length_sequence

    x = LSTM(8)(ip)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(NB_CLASS, activation='softmax')(x)

    model = Model(ip, out)

    model.summary()

    # add load model code here to fine-tune

    return model